## Compute distribution of scores and filter data within score ranges

In [ ]:
import json
import numpy

In [ ]:
with open('/home/mroemmele/elaboration/eval_items/spacy_word_prob_sampled/cohesion_scores_len3.json') as f:
    score_data = json.load(f)

In [ ]:
min_tokens = 3
max_tokens = 3
max_stop_words = 1
min_score = hard_range[0]
max_score = hard_range[1]

qualified_items = []
qualified_scores = []

for idx, item in enumerate(score_data):
#     import pdb;pdb.set_trace()
    # Check duplicate tokens
    tokens_are_unique = len(item['tokens']) == len(set(item['tokens']))

    n_tokens = len(item['tokens'])
    are_qualified_tokens = numpy.all(~numpy.array(
        [values for attr, values in item.items()
         if attr in ('is_spacy_oov', 'is_bert_oov', 'is_punct',
                     'is_space', 'is_digit', 'is_entity')]))
    n_stop_words = sum(item['is_stop'])
    
    tokens_are_eligible = (are_qualified_tokens &
                           tokens_are_unique &
                           (min_tokens <= n_tokens <= max_tokens) &
                           (n_stop_words <= max_stop_words))
    
    score = numpy.array(item['scores']).mean()
    is_in_score_range = min_score <= score <= max_score


    if tokens_are_eligible & is_in_score_range:
        #print("qualified:", item['tokens'], score)
        qualified_items.append(" ".join(item['tokens']))
        qualified_scores.append(score)
        
#     if idx == 100000:
#         break
print(len(qualified_items), "qualified items")

In [ ]:
qualified_items

In [ ]:
filename = "/home/mroemmele/elaboration/eval_items/bookcorpus_test_no_quotes/hard/mintok{}_maxtok{}_maxstop{}_minscore{}_maxscore{}.txt".format(
min_tokens, max_tokens, max_stop_words, min_score, max_score)

with open(filename, 'w') as f:
    f.write("\n".join(qualified_items))
    

### Score distribution stats

In [ ]:
hard_range = (numpy.percentile(qualified_scores, 0), numpy.percentile(qualified_scores, 5))
medium_range = (numpy.percentile(qualified_scores, 49.5), numpy.percentile(qualified_scores, 50.5))
easy_range = (numpy.percentile(qualified_scores, 99.5), numpy.percentile(qualified_scores, 100))

easy_range, medium_range, hard_range

In [ ]:
hard_range = (numpy.percentile(qualified_scores, 0), numpy.percentile(qualified_scores, 10))
medium_range = (numpy.percentile(qualified_scores, 45), numpy.percentile(qualified_scores, 55))
easy_range = (numpy.percentile(qualified_scores, 90), numpy.percentile(qualified_scores, 100))

easy_range, medium_range, hard_range

## Apply filter to existing generated inputs/sentences

In [ ]:
import os
import spacy
from profanity_filter import ProfanityFilter

spacy_model = spacy.load('en')
profanity_filter = ProfanityFilter(nlps={'en': spacy_model})
spacy_model.add_pipe(profanity_filter.spacy_component, last=True)

In [ ]:
input_items_dir = "/home/mroemmele/elaboration/eval_items/bookcorpus_test_no_quotes/input_items/"
generated_sents_dir = "/home/mroemmele/elaboration/eval_items/bookcorpus_test_no_quotes/generated/"

In [ ]:
with open(os.path.join(input_items_dir, 'PG_input_items/hard-mintok3_maxtok3_maxstop1_minscore8.540907714404966e-08_maxscore8.156606483566975e-06.txt')) as f:
    orig_input_words = [line.strip().split(" ") for line in f]
    
with open(os.path.join(generated_sents_dir, 'PG_input_items/hard.txt')) as f:
    orig_gen_sents = [line.strip().split("\t") for line in f]
    
list(zip(orig_input_words, orig_gen_sents))[:2]

In [ ]:
filtered_input_words = []
filtered_gen_sents = []
for idx, (input_words, gen_sents) in enumerate(zip(orig_input_words, orig_gen_sents)):
    # There must be 5 generated sentences
    if len(gen_sents) < 5:
        print("Too few generated sentences in item num {}: {}".format(idx + 1, gen_sents))
        continue
#     # Remove items where at least one sentence is profane
#     contains_profane_sent = False
#     for sent in gen_sents:
#         spacy_sent = spacy_model(sent)
#         if spacy_sent._.is_profane:
#             print("profane sentence for input item {}: {}".format(idx, sent))
#             contains_profane_sent = True
#             break
#     if not contains_profane_sent:
    filtered_input_words.append(input_words)
    filtered_gen_sents.append(gen_sents)

In [ ]:
len(orig_input_words), len(filtered_input_words), len(orig_gen_sents), len(filtered_gen_sents)

In [ ]:
filtered_gen_sents[-2:], filtered_input_words[-2:]

In [ ]:
with open(os.path.join(input_items_dir, 'PG_input_items_added_filter/hard.txt'), 'w') as f:
    f.write("\n".join([" ".join(words) for words in filtered_input_words]))

In [ ]:
with open(os.path.join(generated_sents_dir, 'PG_input_items_added_filter/hard.txt'), 'w') as f:
    f.write("\n".join(["\t".join(sents) for sents in filtered_gen_sents]))